## Lab 9: Isentropic Potential Vorticity Anomalies
In this week's lab, we will use archived GFS model data to create vertical cross-sections to assist in analyzing IPV anomalies.

<br />


### Useful Documentation
1. Xarray open_dataset:  https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html
2. Matplotlib Contour: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contour.html
3. Datetime: https://docs.python.org/3/library/datetime.html
4. Scipy Gaussian Filter: https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html
5. MetPy Potential Temperature: https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.potential_temperature.html




<br />

## Tutorial Part I


In this week's lab, we will create vertical cross-sections of potential temperature and the meridional wind.  In this week's tutorial, I am going to show you how to create a simple vertical cross-section across the United States at 45°N.

<br />

1. First, we open a sample dataset with pressure as the vertical coordinate, here representing a GFS analysis from February 9th, 2023.


In [ ]:
from datetime import datetime
import xarray as xr

time = datetime(2023,2,9,0)
data_location = "/data/AtmSci360/Synp2/Lab_8/"
file_name = f"{time:%m%d%y_%H}_gfs.grib2"

data = xr.open_dataset(f"{data_location}{file_name}", engine='cfgrib', filter_by_keys={'typeOfLevel': 'isobaricInhPa'})
data

<br />

2. Next, we calculate potential temperature and save it back to the data xarray.

In [ ]:
import metpy.calc as calc
pt = calc.potential_temperature(data.isobaricInhPa, data.t)
data = data.assign(pt=pt)

<br />

3. We are now ready to slice the data so that it represents a single vertical cross-section.  This requires two slice operations: first selecting the points at 45&deg;N latitude, then retaining only those between 115&deg;W and 75&deg;W longitude.  

In [ ]:
cross = data.sel(latitude=45)
cross = cross.sel(longitude = slice(360-115,360-75))
cross

<br />

4.  Finally, we are ready to create our plot.  We use matplotlib's contour functionality to do so, noting that we do not need to provide a coordinate transform since we are not plotting data on a map.  In this case, our x-axis is longitude and our y-axis is the natural logarithm of pressure.  I've added comments below so you can better follow along.

In [ ]:
"""
Import the usual packages for plotting.
"""
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import numpy as np

"""
Generate a blank figure and subplot.  Note we do not need to provide a projection since we are not
plotting on a map.
"""
fig = plt.figure(figsize=(16,9), dpi=300)
ax = plt.subplot()

"""
The contour statement, indicating longitude at the x-axis variable and pressure as the y-axis variable.
Gaussian-smoothed potential temperature is what is plotted in labeled contours with a 5 K contour interval.
"""
cont = plt.contour(cross.longitude-360, cross.isobaricInhPa.values, gaussian_filter(cross.pt.values,1), np.arange(230,900,5))
ax.clabel(cont, cont.levels, inline=True, fontsize=6)

"""
Set the y-axis as natural logarithmic.
"""
ax.set_yscale('symlog')

"""
Set the y-axis limits to 1000 hPa at the bottom and 100 hPa at the top.
"""
plt.ylim(1000,100)

"""
This changes the tick labels.  By default, matplotlib uses scientific notation for natural logarithm axis labels.
Manually specifying the tick locations and their labels addresses this issue.
"""
ax.set_yticks(np.arange(1000, 50, -100))
ax.set_yticklabels(np.arange(1000, 50, -100))

### Instructions
So that you can complete this week's lab, I have locally archived the March 7th, 2023 1200 UTC GFS analysis.  The location of these data is provided for you below, as is the filename convention if you wish to use a datetime object to select your time of interest.

Create the following maps and plots for March 7th, 2023 at 1200 UTC:

<ul>
    <li>Vertical cross-section between 1000-100 hPa of potential temperature and the meridional wind at 45&deg;N between 145&deg;W and 80&deg;W</li>
    <li>Vertical cross-section between 1000-100 hPa of potential temperature and the meridional wind at 25&deg;N between 120&deg;W and 85&deg;W</li>
</ul>
    
Be sure your maps follow the "good map" guidelines, and do not forget to import the necessary packages before you start coding.

In [ ]:
data_location = "/data/AtmSci360/Synp2/Lab_9/"
file_name = f"{time:%m%d%y_%H}_gfs.grib2"